# Dataset AdventureWorksDW

### Operações SQL simples

A base **AdventureWorksDW** https://github.com/microsoft/sql-server-samples/ é bastante conhecida no mundo de dados Microsoft.

Vamos fazer algumas atividades com o SparkSQL, como desafios.

### Carregando o PySpark

In [1]:
# !pip install pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [3]:
conf = SparkConf().setMaster('local').setAppName('PySpark SQL')
sc = SparkContext.getOrCreate(conf = conf)

Criação do contexto do objeto SparkSQL que será responsável por executar as *query* do Spark com comandos SQL.

In [4]:
sql = SQLContext(sc)

In [5]:
sql

Aqui é a criação de um Dataframe com os dados que estamos lendo do arquivo CSV. Usamos o contexto do SparkSQL, mas ainda sim é um Dataframe

In [6]:
FactInternetSales_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/FactInternetSales.csv')
DimSalesTerritory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimSalesTerritory.csv')
DimProductSubcategory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProductSubcategory.csv')
DimProductCategory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProductCategory.csv')
DimProduct_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProduct.csv')
DimCustomer_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimCustomer.csv')

In [7]:
FactInternetSales_Spark.printSchema()

root
 |-- ProductKey: string (nullable = true)
 |-- OrderDateKey: string (nullable = true)
 |-- DueDateKey: string (nullable = true)
 |-- ShipDateKey: string (nullable = true)
 |-- CustomerKey: string (nullable = true)
 |-- PromotionKey: string (nullable = true)
 |-- CurrencyKey: string (nullable = true)
 |-- SalesTerritoryKey: string (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- SalesOrderLineNumber: string (nullable = true)
 |-- RevisionNumber: string (nullable = true)
 |-- OrderQuantity: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- ExtendedAmount: string (nullable = true)
 |-- UnitPriceDiscountPct: string (nullable = true)
 |-- DiscountAmount: string (nullable = true)
 |-- ProductStandardCost: string (nullable = true)
 |-- TotalProductCost: string (nullable = true)
 |-- SalesAmount: string (nullable = true)
 |-- TaxAmt: string (nullable = true)
 |-- Freight: string (nullable = true)
 |-- CarrierTrackingNumber: string (nullable = tru

In [8]:
FactInternetSales_Spark.select(["ProductKey","TotalProductCost"]).show(10)

+----------+----------------+
|ProductKey|TotalProductCost|
+----------+----------------+
|       310|       2171.2942|
|       346|       1912.1544|
|       346|       1912.1544|
|       336|        413.1463|
|       346|       1912.1544|
|       311|       2171.2942|
|       310|       2171.2942|
|       351|       1898.0944|
|       344|       1912.1544|
|       312|       2171.2942|
+----------+----------------+
only showing top 10 rows



A partir do dataframe **dadosSpark**, vamos registrar uma tabela temporária do SQL, chamada **Carros**

In [9]:
FactInternetSales_Spark.registerTempTable("FactInternetSales")
DimSalesTerritory_Spark.registerTempTable("DimSalesTerritory")
DimProductSubcategory_Spark.registerTempTable("DimProductSubcategory")
DimProductCategory_Spark.registerTempTable("DimProductCategory")
DimProduct_Spark.registerTempTable("DimProduct")
DimCustomer_Spark.registerTempTable("DimCustomer")

A partir do motor do **SparkSQL** vamos escrever uma *query* em SQL que retornará todas as linhas e colunas da nossa base

In [10]:
sql.sql("SELECT sub.* FROM \
            DimProductSubcategory as SUB inner join \
            DimProductCategory Cat \
                on SUB.ProductCategoryKey = Cat.ProductCategoryKey \
            where Cat.ProductCategoryKey = 2").show()

+---------------------+------------------------------+-----------------------------+-----------------------------+----------------------------+------------------+
|ProductSubcategoryKey|ProductSubcategoryAlternateKey|EnglishProductSubcategoryName|SpanishProductSubcategoryName|FrenchProductSubcategoryName|ProductCategoryKey|
+---------------------+------------------------------+-----------------------------+-----------------------------+----------------------------+------------------+
|                    4|                             4|                   Handlebars|                        Barra|               Barre d'appui|                 2|
|                    5|                             5|              Bottom Brackets|              Eje de pedalier|             Axe de p�dalier|                 2|
|                    6|                             6|                       Brakes|                       Frenos|                      Freins|                 2|
|                    7

In [21]:
# FactInternetSales_Spark.select(["ProductKey","ExtendedAmount"])\
#                                 .show(10)
sql.sql("SELECT * FROM FactInternetSales")\
.show()

+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+--------------+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+---------------------+----------------+---------------+--------------+-------------+
|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|CustomerKey|PromotionKey|CurrencyKey|SalesTerritoryKey|SalesOrderNumber|SalesOrderLineNumber|RevisionNumber|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount|  TaxAmt|Freight|CarrierTrackingNumber|CustomerPONumber|      OrderDate|       DueDate|     ShipDate|
+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+--------------+-------------+---------+--------------+--------------------+--------------+-----

In [22]:
# sql.sql("").show()
sql.sql("SELECT ProductKey AS ProdId FROM FactInternetSales\
        GROU")\
        .show()

+------+
|ProdId|
+------+
|   310|
|   346|
|   346|
|   336|
|   346|
|   311|
|   310|
|   351|
|   344|
|   312|
|   312|
|   330|
|   313|
|   314|
|   314|
|   311|
|   310|
|   311|
|   314|
|   311|
+------+
only showing top 20 rows



In [23]:
# sql.sql("").show()
sql.sql("select CatP.EnglishProductCategoryName as Categoria, SalT.SalesTerritoryCountry as Pais, round(sum(FIS.SalesAmount),2) AS TotalVendas from \
    FactInternetSales FIS inner join DimProduct AS Prod on FIS.ProductKey = Prod.ProductKey \
    inner join DimProductSubcategory AS SubC on Prod.ProductSubcategoryKey = SubC.ProductSubcategoryKey \
    inner join DimProductCategory AS CatP on SubC.ProductCategoryKey = CatP.ProductCategoryKey \
    inner join DimSalesTerritory AS SalT on FIS.SalesTerritoryKey = SalT.SalesTerritoryKey \
    group by CatP.EnglishProductCategoryName, SalT.SalesTerritoryCountry \
    order by SalT.SalesTerritoryCountry, CatP.EnglishProductCategoryName").show(10)

+-----------+---------+-----------+
|  Categoria|     Pais|TotalVendas|
+-----------+---------+-----------+
|Accessories|Australia|  138690.63|
|      Bikes|Australia|  8852050.0|
|   Clothing|Australia|   70259.95|
|Accessories|   Canada|  103377.85|
|      Bikes|   Canada| 1821302.39|
|   Clothing|   Canada|   53164.62|
|Accessories|   France|   63406.78|
|      Bikes|   France| 2553575.71|
|   Clothing|   France|   27035.22|
|Accessories|  Germany|   62232.59|
+-----------+---------+-----------+
only showing top 10 rows



In [ ]:
# sql.sql("").show()